In [11]:
import findspark
findspark.init('/home/ubuntu/spark-2.2.0-bin-hadoop2.7')
from pyspark.sql import SparkSession

import os
os.getcwd()
data_folder = '/home/ubuntu/data/raw'
os.chdir(data_folder)
os.getcwd()

'/home/ubuntu/data/raw'

In [4]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [26]:
file_name = '/sample_linear_regression_data.txt'

all_data = spark.read.format('libsvm').load(data_folder+file_name)

In [27]:
all_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [57]:
row = all_data[0]
row.asDict()

TypeError: 'Column' object is not callable

In [33]:
#Split Data
train_data, test_data = all_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                374|
|   mean| 0.2404145401049108|
| stddev|  10.62258198111317|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [34]:
#Create instance of model
lr = LinearRegression(featuresCol='features', labelCol='label',
                     predictionCol = 'prediction')

In [35]:
#Fit Model
lrModel = lr.fit(train_data)

In [30]:
#Get data from model
print(lrModel.coefficients)
lrModel.intercept


[0.666891601007,0.33954753076,-0.902681917323,2.67840101518,0.239202071629,1.28902057922,-0.104525648542,-1.28287490953,-1.98438496491,-0.0215246635331]


0.24645993033071842

In [31]:
training_summary = lrModel.summary
training_summary.rootMeanSquaredError

10.189335323159474

In [38]:
#MOdel Evaluation
test_results = lrModel.evaluate(test_data)
test_results.rootMeanSquaredError

9.427606888873248

In [58]:
unlabled_data = test_data.select('features')

In [62]:
unlabled_data.show(5)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows



In [64]:
predictions = lrModel.transform(unlabled_data)
predictions.show(5)

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| -4.330262021644618|
|(10,[0,1,2,3,4,5,...|-1.6191109239717214|
|(10,[0,1,2,3,4,5,...|  4.077883238247792|
|(10,[0,1,2,3,4,5,...|0.09459940195331801|
|(10,[0,1,2,3,4,5,...| -2.290473944686299|
+--------------------+-------------------+
only showing top 5 rows

